In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# Генерация данных
def generate_data(func, n_samples=100):
    x = torch.linspace(0, 1, n_samples).unsqueeze(1)  # Входные данные
    y = func(x)  # Значения функции
    return x, y

# Определение функций
def cos_func(x):
    return torch.cos(x)

def exp_func(x):
    return torch.exp(x)

In [21]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Sigmoid()
        self.output_layer = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

In [22]:
# Обучение модели
def train_model(model, criterion, optimizer, x_train, y_train, epochs):
    for epoch in range(epochs):
        # Прямой проход
        predictions = model(x_train)
        loss = criterion(predictions, y_train)

        # Обратное распространение и обновление весов
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Печать потерь
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.6f}')

In [23]:
def integrate_model(model, n_samples=100):
    x = torch.linspace(0, 1, n_samples).unsqueeze(1)  # Точки для интеграции
    with torch.no_grad():
        y = model(x)
    integral = torch.trapz(y.squeeze(), x.squeeze())  # Метод трапеций
    return integral.item()

In [24]:
def integrate_model_analytically(model):
    # Извлечение параметров
    with torch.no_grad():
        W1 = model.input_layer.weight  # W^{(1)}
        b1 = model.input_layer.bias    # b^{(1)}
        W2 = model.output_layer.weight  # W^{(2)}
        b2 = model.output_layer.bias    # b^{(2)}

    # Сигмоида
    def sigmoid(x):
        return 1 / (1 + torch.exp(-x))

    def sigmoid_integral(a, b, x_min, x_max):
        """
        Вычисляет аналитический интеграл сигмоиды: ∫ σ(ax + b) dx
        """
        if a == 0:
            raise ValueError("Параметр 'a' не может быть равен нулю для корректного вычисления.")
        
        # Преобразуем параметры в тензоры
        a = torch.tensor(a, dtype=torch.float32)
        b = torch.tensor(b, dtype=torch.float32)
        x_min = torch.tensor(x_min, dtype=torch.float32)
        x_max = torch.tensor(x_max, dtype=torch.float32)
        
        # Вычисляем аналитический интеграл
        return (1 / a) * (
            torch.log(1 + torch.exp(a * x_max + b)) - torch.log(1 + torch.exp(a * x_min + b))
        )

    # Интеграл: ∫ f(X) dX = ∫ (b^{(2)} + W^{(2)T} φ(b^{(1)} + W^{(1)}X)) dX
    x_min, x_max = 0, 1  # Пределы интегрирования
    integral = b2.item() * (x_max - x_min)  # Член b^{(2)}

    for i in range(W2.size(1)):  # По скрытым нейронам
        for j in range(W1.size(1)):  # По входным переменным
            a = W1[i, j].item()
            b = b1[i].item()
            weight = W2[0, i].item()
            integral += weight * sigmoid_integral(a, b, x_min, x_max)

    return integral

## 1D exp[x]

In [54]:
# Гиперпараметры
input_size = 1  # Одна переменная x
hidden_size = 10  # Количество нейронов в скрытом слое
learning_rate = 0.01
num_epochs = 10000

# Создаем модель
model = MLP(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [55]:
# Генерация данных
x_train, y_train = generate_data(exp_func, 10000)  # Используем cos(x) или exp_func

In [56]:
# Обучение модели
train_model(model, criterion, optimizer, x_train, y_train, num_epochs)

Epoch [100/10000], Loss: 0.128916
Epoch [200/10000], Loss: 0.059543
Epoch [300/10000], Loss: 0.015104
Epoch [400/10000], Loss: 0.004839
Epoch [500/10000], Loss: 0.003649
Epoch [600/10000], Loss: 0.003084
Epoch [700/10000], Loss: 0.002617
Epoch [800/10000], Loss: 0.002222
Epoch [900/10000], Loss: 0.001885
Epoch [1000/10000], Loss: 0.001593
Epoch [1100/10000], Loss: 0.001338
Epoch [1200/10000], Loss: 0.001114
Epoch [1300/10000], Loss: 0.000917
Epoch [1400/10000], Loss: 0.000747
Epoch [1500/10000], Loss: 0.000600
Epoch [1600/10000], Loss: 0.000475
Epoch [1700/10000], Loss: 0.000372
Epoch [1800/10000], Loss: 0.000288
Epoch [1900/10000], Loss: 0.000221
Epoch [2000/10000], Loss: 0.000169
Epoch [2100/10000], Loss: 0.000130
Epoch [2200/10000], Loss: 0.000101
Epoch [2300/10000], Loss: 0.000080
Epoch [2400/10000], Loss: 0.000066
Epoch [2500/10000], Loss: 0.000056
Epoch [2600/10000], Loss: 0.000048
Epoch [2700/10000], Loss: 0.000043
Epoch [2800/10000], Loss: 0.000040
Epoch [2900/10000], Loss: 0.0

In [57]:
# Вычисление интеграла
integral = integrate_model(model)
print(f'Приближенное значение интеграла: {integral:.6f}')

Приближенное значение интеграла: 1.718310


In [58]:
# Для сравнения: точное значение интеграла
true_integral = np.exp(1) - np.exp(0)  # Для cos(x)
print(f'Точное значение интеграла: {true_integral.item():.6f}')

Точное значение интеграла: 1.718282


In [59]:
# Вычисление интеграла аналитически
integral_analytical = integrate_model_analytically(model)
print(f'Аналитическое значение интеграла нейросети: {integral_analytical:.6f}')

Аналитическое значение интеграла нейросети: 1.718296


## 1D cos(x)

In [60]:
# Создаем модель
model = MLP(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [61]:
# Генерация данных
x_train, y_train = generate_data(cos_func, 10000)  # Используем cos(x) или exp_func

In [62]:
# Обучение модели
train_model(model, criterion, optimizer, x_train, y_train, num_epochs)

Epoch [100/10000], Loss: 0.007816
Epoch [200/10000], Loss: 0.002053
Epoch [300/10000], Loss: 0.001363
Epoch [400/10000], Loss: 0.001294
Epoch [500/10000], Loss: 0.001235
Epoch [600/10000], Loss: 0.001180
Epoch [700/10000], Loss: 0.001129
Epoch [800/10000], Loss: 0.001082
Epoch [900/10000], Loss: 0.001039
Epoch [1000/10000], Loss: 0.000999
Epoch [1100/10000], Loss: 0.000959
Epoch [1200/10000], Loss: 0.000920
Epoch [1300/10000], Loss: 0.000881
Epoch [1400/10000], Loss: 0.000840
Epoch [1500/10000], Loss: 0.000796
Epoch [1600/10000], Loss: 0.000749
Epoch [1700/10000], Loss: 0.000697
Epoch [1800/10000], Loss: 0.000639
Epoch [1900/10000], Loss: 0.000576
Epoch [2000/10000], Loss: 0.000507
Epoch [2100/10000], Loss: 0.000434
Epoch [2200/10000], Loss: 0.000358
Epoch [2300/10000], Loss: 0.000282
Epoch [2400/10000], Loss: 0.000211
Epoch [2500/10000], Loss: 0.000149
Epoch [2600/10000], Loss: 0.000098
Epoch [2700/10000], Loss: 0.000061
Epoch [2800/10000], Loss: 0.000036
Epoch [2900/10000], Loss: 0.0

In [63]:
# Вычисление интеграла
integral = integrate_model(model)
print(f'Приближенное значение интеграла: {integral:.6f}')

Приближенное значение интеграла: 0.841498


In [65]:
# Для сравнения: точное значение интеграла
true_integral = np.sin(1) - np.sin(0)  # Для cos(x)
print(f'Точное значение интеграла: {true_integral.item():.6f}')

Точное значение интеграла: 0.841471


In [66]:
# Вычисление интеграла аналитически
integral_analytical = integrate_model_analytically(model)
print(f'Аналитическое значение интеграла нейросети: {integral_analytical:.6f}')

Аналитическое значение интеграла нейросети: 0.841504


## 2D $sin(x) + cos(y)$

In [84]:
def integrate_model_2d_analytically(model, x_min, x_max, y_min, y_max):
    """
    Интегрирует нейронную сеть с 2 входами по обеим переменным аналитически.
    """
    # Извлечение параметров сети
    with torch.no_grad():
        W1 = model.input_layer.weight  # W^{(1)}: веса входного слоя
        b1 = model.input_layer.bias    # b^{(1)}: смещения входного слоя
        W2 = model.output_layer.weight  # W^{(2)}: веса выходного слоя
        b2 = model.output_layer.bias    # b^{(2)}: смещение выходного слоя

    # Сигмоида и её интеграл
    def sigmoid(x):
        return 1 / (1 + torch.exp(-x))

    def sigmoid_integral(a, b, x_min, x_max):
        """
        Вычисляет аналитический интеграл сигмоиды: ∫ σ(ax + b) dx
        """
        if a == 0:
            raise ValueError("Параметр 'a' не может быть равен нулю для корректного вычисления.")
        
        # Преобразуем все параметры в тензоры, чтобы обеспечить правильную работу с PyTorch
        a = torch.tensor(a, dtype=torch.float32)
        b = torch.tensor(b, dtype=torch.float32)
        x_min = torch.tensor(x_min, dtype=torch.float32)
        x_max = torch.tensor(x_max, dtype=torch.float32)
    
        return (1 / a) * (
            torch.log(1 + torch.exp(a * x_max + b)) - torch.log(1 + torch.exp(a * x_min + b))
        )


    # Интеграл: ∫ f(x, y) dx dy
    integral = b2.item() * (x_max - x_min) * (y_max - y_min)  # Член b^{(2)}

    for i in range(W2.size(1)):  # По скрытым нейронам
        weight = W2[0, i].item()  # W^{(2)}

        # Интегрируем по каждой переменной
        for j_x, j_y in [(0, 0), (0, 1), (1, 0), (1, 1)]:  # Индексы весов W^{(1)}
            a_x = W1[i, j_x].item()  # Вес для x
            a_y = W1[i, j_y].item()  # Вес для y
            b = b1[i].item()         # Смещение

            # Интеграция по y
            inner_integral = sigmoid_integral(a_y, b, y_min, y_max)
            # Интеграция по x
            integral += weight * sigmoid_integral(a_x, inner_integral, x_min, x_max)

    return integral

In [85]:
# Гиперпараметры
input_dim = 2  # Два входа: x и y
hidden_dim = 10  # Размер скрытого слоя
output_dim = 1  # Один выход
learning_rate = 0.01
epochs = 10000

In [86]:
# Создаем модель
model = MLP(input_dim, hidden_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [87]:
def generate_2D_data(num_samples):
    x = torch.rand(num_samples, 1)
    y = torch.rand(num_samples, 1)
    f_values = torch.sin(x) + torch.cos(y)
    return torch.cat((x, y), dim=1), f_values

In [88]:
# Генерация данных
X_train, y_train = generate_2D_data(10000)

In [89]:
for epoch in range(epochs):
    # Генерация данных
    X_train, y_train = generate_data(1000)
    
    # Обучение
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [50/10000], Loss: 0.0813
Epoch [100/10000], Loss: 0.0581
Epoch [150/10000], Loss: 0.0316
Epoch [200/10000], Loss: 0.0182
Epoch [250/10000], Loss: 0.0065
Epoch [300/10000], Loss: 0.0025
Epoch [350/10000], Loss: 0.0014
Epoch [400/10000], Loss: 0.0011
Epoch [450/10000], Loss: 0.0012
Epoch [500/10000], Loss: 0.0011
Epoch [550/10000], Loss: 0.0012
Epoch [600/10000], Loss: 0.0012
Epoch [650/10000], Loss: 0.0011
Epoch [700/10000], Loss: 0.0011
Epoch [750/10000], Loss: 0.0011
Epoch [800/10000], Loss: 0.0011
Epoch [850/10000], Loss: 0.0011
Epoch [900/10000], Loss: 0.0010
Epoch [950/10000], Loss: 0.0011
Epoch [1000/10000], Loss: 0.0011
Epoch [1050/10000], Loss: 0.0011
Epoch [1100/10000], Loss: 0.0011
Epoch [1150/10000], Loss: 0.0011
Epoch [1200/10000], Loss: 0.0010
Epoch [1250/10000], Loss: 0.0010
Epoch [1300/10000], Loss: 0.0010
Epoch [1350/10000], Loss: 0.0010
Epoch [1400/10000], Loss: 0.0011
Epoch [1450/10000], Loss: 0.0010
Epoch [1500/10000], Loss: 0.0010
Epoch [1550/10000], Loss: 0.00

In [90]:
# Пределы интегрирования
x_min, x_max = 0, 1
y_min, y_max = 0, 1

# Вычисление интеграла нейросети
integral_2d_analytical = integrate_model_2d_analytically(model, x_min, x_max, y_min, y_max)
print(f'Аналитическое значение двойного интеграла нейросети: {integral_2d_analytical:.6f}')

Аналитическое значение двойного интеграла нейросети: 3.633325


/var/folders/df/v02b0vfx2tbc5plrdx4v6gk80000gn/T/ipykernel_14564/2003117665.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(b, dtype=torch.float32)


In [91]:
# Значения для cos(1) и sin(1)
cos_1 = np.cos(1)
sin_1 = np.sin(1)

# Аналитический интеграл
true_integral = (1 - cos_1) + sin_1
print(f"Значение аналитического интеграла: {true_integral:.6f}")

Значение аналитического интеграла: 1.301169
